In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import config

In [2]:
# Test set: use config.TEST_DIR so paths work from project root. Class names from config for consistent ordering.
test_set = tf.keras.utils.image_dataset_from_directory(
    str(config.TEST_DIR),
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=32,
    image_size=config.IMG_SIZE,
    shuffle=False,
    interpolation="bilinear"
)
class_name = list(config.CLASS_NAMES)  # same order as training; do not use test_set.class_names
print(f"Loaded {len(class_name)} classes from {config.TEST_DIR}")
print(class_name[:5], "...")

Found 3513 files belonging to 38 classes.


2026-01-29 14:16:00.002614: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2026-01-29 14:16:00.002637: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2026-01-29 14:16:00.002640: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.92 GB
I0000 00:00:1769667360.003081 3232023 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1769667360.003451 3232023 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Sp

In [3]:
# Load model from project root so notebook works from any cwd
model_path = config.PROJECT_ROOT / "trained_plant_disease_model.h5"
cnn = tf.keras.models.load_model(str(model_path))

In [4]:
import cv2
from pathlib import Path

# Use a sample image from data/test so the notebook works without an extra "images" folder.
# Picks first image from Apple___Cedar_apple_rust; override image_path to use your own file.
sample_dir = config.TEST_DIR / "Apple___Cedar_apple_rust"
sample_images = list(sample_dir.glob("*.JPG")) or list(sample_dir.glob("*.jpg"))
image_path = str(sample_images[0]) if sample_images else None

if image_path is None:
    print(f"No images found in {sample_dir}. Add test images to data/test/<class_name>/ or set image_path to your image.")
else:
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to load image at '{image_path}'")
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.title(f"Test Image: {Path(image_path).name}")
        plt.xticks([])
        plt.yticks([])
        plt.show()
        print(f"Loaded: {image_path}")


Error: Unable to load image at 'images/AppleCedarRust1.JPG'


[ WARN:0@2.270] global loadsave.cpp:268 findDecoder imread_('images/AppleCedarRust1.JPG'): can't open/read file: check file path/integrity


In [5]:
# Run prediction only if we have a valid image_path (from previous cell)
if 'image_path' not in dir() or image_path is None:
    print("Run the previous cell first to load a sample image from data/test.")
else:
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=config.IMG_SIZE)
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr], dtype=np.float32) / 255.0  # normalize to [0,1] to match canonical pipeline
    predictions = cnn.predict(input_arr, verbose=0)
    result_index = int(np.argmax(predictions))
    print("Predicted class index:", result_index)
    print("Predicted class:", class_name[result_index])



FileNotFoundError: [Errno 2] No such file or directory: 'images/AppleCedarRust1.JPG'

In [ ]:
# Display the image with predicted label (run previous cells first so img and result_index exist)
if 'img' in dir() and 'result_index' in dir() and 0 <= result_index < len(class_name):
    model_prediction = class_name[result_index]
    plt.imshow(img)
    plt.title(f"Predicted: {model_prediction}")
    plt.xticks([])
    plt.yticks([])
    plt.show()
else:
    print("Run the previous cells to get img and result_index.")